In [308]:
import bs4
import os
import json
import requests

url = "http://books.toscrape.com/"

In [309]:
request_data = bs4.BeautifulSoup(requests.get(url).content)

In [339]:
category_list = request_data.find('ul',{'class':'nav nav-list'})

In [340]:
category_list = category_list.findAll('li')
#len(category_list)

In [330]:
category_list_dup = category_list.copy()
category_list.clear()
for item in category_list_dup[1:]:
    item_dic = {
        'name': item.text.strip(),
        'href':"https://books.toscrape.com/" + item.find('a')['href']
    }
    category_list.append(item_dic)

In [331]:
path = os.getcwd()
for item in category_list:
    os.mkdir("{}\\{}".format(path,item['name']))

In [337]:
for category in category_list:
    category_data = bs4.BeautifulSoup(requests.get(category['href']).content)
    
    
        
    books = category_data.findAll('div',{'class':'image_container'})
    book_urls = []
    for book in books:
        book_url = book.find('a')['href'][9:]
        book_urls.append(book_url)
   
    for book_url in book_urls:
        book_data = bs4.BeautifulSoup(requests.get("http://books.toscrape.com/catalogue/{}".format(book_url)).content)
        
        book_name = book_data.find('h1').text
        restricted_char = '*/\:?|<>"'
        
        for char in restricted_char:
            book_name = book_name.replace(char,'')
            
        #print(book_name)
        book_name = book_name[0:186]
        
        try:
            os.mkdir("{}\\{}\\{}".format(os.getcwd(),category['name'],book_name))
        except:
            continue
        
        img_url = "http://books.toscrape.com/" + book_data.find('div',{'class':'item active'}).find('img')['src'][6:]
        
        img_data = requests.get(img_url).content
        with open("./{}/{}/image.jpg".format(category['name'],book_name),'wb') as file:
            file.write(img_data)
        
        
        data = {
            'name': book_data.find('h1').text,
            'price': book_data.find('p',{'class':'price_color'}).text,
            'UPC':book_data.find('table').findAll('td')[0].text,
            'tax':book_data.find('table').findAll('td')[4].text,
            'availabillity':book_data.find('table').findAll('td')[5].text,
            'reviews':book_data.find('table').findAll('td')[6].text,
            'description':book_data.findAll('p')[3].text
        }
        
        data = json.dumps(data)
        
        with open("./{}/{}/data.txt".format(category['name'],book_name),'w') as file:
            file.write(data)